# IMDB: recursive neural networks

## Data preprocessing

### Required imports

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Processing

Load the training and test data.

In [2]:
(x_train, y_train), (x_test, y_test) = imdb.load_data()

Since the review vary in length, and we prefer to limit the computation time, we will base the classification on the first 100 features of each input sequence.

In [3]:
feature_length = 100
x_train = sequence.pad_sequences(x_train, maxlen=feature_length)
x_test = sequence.pad_sequences(x_test, maxlen=feature_length)

Again, to limit computation time, we restrict the number of features to 5000.  This means we map all features with an index of 20000 or more to 0.

In [4]:
nr_features = 5_000
x_train[x_train > nr_features - 1] = 0
x_test[x_test > nr_features - 1] = 0

Now the training and test input are 2D arrays. We split the training set into a subset for actual training, and one for validation.  First we seed the random number generator to ensure reproducibility. In this case, we will use part of the 25000 test examples as valiation data.

In [5]:
np.random.seed(1234)

In [6]:
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test)

## GRU

### Required imports & model definition

In [7]:
from keras.layers import Activation, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.models import Sequential
from keras.optimizers import Adam

Again, to limit training times, we restrict ourselfs to using a limited number of features.

In [8]:
model = Sequential()
model.add(Embedding(nr_features, 64, mask_zero=True,
                    input_length=feature_length))
model.add(GRU(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           320000    
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 344,833
Trainable params: 344,833
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='binary_crossentropy', optimizer=Adam(),
              metrics=['accuracy'])

###    Training

In [11]:
history = model.fit(x_train, y_train, batch_size=64, epochs=10,
                    validation_data=(x_val, y_val))

Train on 25000 samples, validate on 6250 samples
Epoch 1/10
25000/25000 [==============================] - 71s 3ms/step - loss: 0.4535 - acc: 0.7750 - val_loss: 0.3570 - val_acc: 0.8414
Epoch 2/10
25000/25000 [==============================] - 69s 3ms/step - loss: 0.3060 - acc: 0.8751 - val_loss: 0.3766 - val_acc: 0.8317
Epoch 3/10
25000/25000 [==============================] - 69s 3ms/step - loss: 0.2580 - acc: 0.8977 - val_loss: 0.3574 - val_acc: 0.8493
Epoch 4/10
25000/25000 [==============================] - 69s 3ms/step - loss: 0.2153 - acc: 0.9182 - val_loss: 0.4019 - val_acc: 0.8405
Epoch 5/10
25000/25000 [==============================] - 70s 3ms/step - loss: 0.1744 - acc: 0.9365 - val_loss: 0.4157 - val_acc: 0.8414
Epoch 6/10
25000/25000 [==============================] - 69s 3ms/step - loss: 0.1435 - acc: 0.9491 - val_loss: 0.5269 - val_acc: 0.8302
Epoch 7/10
25000/25000 [==============================] - 69s 3ms/step - loss: 0.1171 - acc: 0.9593 - val_loss: 0.5158 - val_acc:

The training accuracy is much better than the validation accurcy, so the model is likely heavily overtrained.

### Testing

In [12]:
model.evaluate(x_test, y_test)

18750/18750 [==============================] - 11s 565us/step


[0.6996061566193899, 0.8343466666857402]

## LSTM

### Required imports & model definition

In [13]:
from keras.layers.recurrent import LSTM

Again, to limit training times, we restrict ourselfs to using a limited number of features.

In [14]:
model = Sequential()
model.add(Embedding(nr_features, 64, mask_zero=True,
                    input_length=feature_length))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 64)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 353,089
Trainable params: 353,089
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='binary_crossentropy', optimizer=Adam(),
              metrics=['accuracy'])

###    Training

In [17]:
history = model.fit(x_train, y_train, batch_size=64, epochs=10,
                    validation_data=(x_val, y_val))

Train on 25000 samples, validate on 6250 samples
Epoch 1/10
25000/25000 [==============================] - 98s 4ms/step - loss: 0.4387 - acc: 0.7948 - val_loss: 0.3717 - val_acc: 0.8365
Epoch 2/10
25000/25000 [==============================] - 97s 4ms/step - loss: 0.3045 - acc: 0.8735 - val_loss: 0.3505 - val_acc: 0.8520
Epoch 3/10
25000/25000 [==============================] - 96s 4ms/step - loss: 0.2594 - acc: 0.8978 - val_loss: 0.3841 - val_acc: 0.8424
Epoch 4/10
25000/25000 [==============================] - 96s 4ms/step - loss: 0.2194 - acc: 0.9138 - val_loss: 0.4189 - val_acc: 0.8370
Epoch 5/10
25000/25000 [==============================] - 96s 4ms/step - loss: 0.1832 - acc: 0.9308 - val_loss: 0.4222 - val_acc: 0.8358
Epoch 6/10
25000/25000 [==============================] - 98s 4ms/step - loss: 0.1572 - acc: 0.9404 - val_loss: 0.4799 - val_acc: 0.8307
Epoch 7/10
25000/25000 [==============================] - 95s 4ms/step - loss: 0.1329 - acc: 0.9524 - val_loss: 0.5651 - val_acc:

The training accuracy is much better than the validation accurcy, so the model is likely heavily overtrained.

### Testing

In [18]:
model.evaluate(x_test, y_test)

18750/18750 [==============================] - 12s 637us/step


[0.5942397989654541, 0.822453333307902]